# SquamataAeroRAD - Jupyter notebook for creating metadata for Aeromagnetic data-releases to ScienceBase

### Version 1.8

#### Visit https://github.com/pbrown-usgs/SquamataAeroMag/blob/master/SquamataAeroRadEntityAndAttributeArchive.ipynb for Entity and Attribute code from previous Airborne Radiometric surveys

This module performs the following operations:
- Create list of data directories.
- Identify files accompanying data release.
- Create file listing for metadata XML markup.
- Identify and load Aeromagnetic *.CSV data file.
- Create file list of data release files
- Create User Editable Keywords Listing
- Create entity and attribute XML markup for all files being released.
- Poplulate metadata template
- Validate metadata; create error log; create HTML and FGDC Text versions of the metadata. (In development - use https://mrdata.usgs.gov/validation/ for validation in the interim)

Known issues needing repair:
- Add items here...

## Future development plans for SquamataSB

- Create all child metadata files from first example created in previous steps. 
- Batch upload files to ScienceBase.
- Batch remove files from ScienceBase. 
- Change ScienceBase parameters such as citation information, add orcid ids, add USGS CMS tags, etc. 

### Instructions
- Create a template xml format that contains boilerplate text common to all childeren in a data release.  Be sure this template contains the approriate curly bracket tags, {SquamataTagExample} used to populate the template using SquamataAeromag.

### To execute a function/command select a cell and Hold-Shift + Press-Enter

**The 'r' signifies a string literal. Use for paths.**

Metadata wizard:  Advanced, Open In a jupyter Notebook?
Metadata Wizard 2.o from ScienceBase

v1.0: Read csv File into pandas
v1.1: Create Array of csv column headers, assign header array of channel descriptions and units, create entity and attribute text for csv file
v1.2 Create file listing, create editable keywords text
v1.3 Creat entity and attributes for files other than CSV
v1.4 Populate metadata template
v1.5 Populate variables with boilerplate text, create file lists, write to metadata template.
v1.6 and 1.7 - Add Raw Up and Down spec file EandA
v1.8 Add Std Order


In [1]:
# Phil Brown (pbrown@usgs.gov) 2019 Beta
# Working Python 3 Notebook used to facilitate the release of Audio Magnetotelluric (AMT) Data to ScienceBase.

In [2]:
# Test Cell
print ("Jupyter is working.") #To run this cell, hold down Shift and press Enter.

Jupyter is working.


In [3]:
# Load required Libraries
import sys
import os
import zipfile
import csv
#import pysb
import requests
import shutil
from shutil import copyfile
import zipfile
import datetime
import glob
from lxml import etree
import json
import pickle
import shutil
import fileinput
import json
import pandas as pd
import numpy as np
from IPython.core.display import display
from IPython.core.display import HTML
from lxml import etree
##from pymdwizard.core.xml_utils import XMLRecord
##from pymdwizard.core.xml_utils import XMLNode
import re
from ipywidgets import *
from IPython.display import display
from IPython.html.widgets import widgets
import datetime
import dateutil.parser
import time
from IPython.display import Javascript

C:\ProgramData\Anaconda3\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Collect Information For Metadata Template

In [4]:
# Create an Array of Origin Names between single quotes below
arrOrigin = []

#Change Text Below to the Appropriate Data Release Creators - seperate list items in single quotes ('')
# seperated by a comma (,).
#########################################################################################################################
arrOrigin = ['McCafferty, A.E.', 'Brown, P.J.'] 
#########################################################################################################################
origin = ''
for i in range(len(arrOrigin)):
    origin = origin + '\n\t\t\t\t<origin>' + arrOrigin[i] + '</origin>'

print ('Origin text:') 
print (origin)    

Origin text:

				<origin>McCafferty, A.E.</origin>
				<origin>Brown, P.J.</origin>


### -- Replace with month, day, year (YYYYMMDD) you request SB page to go live --

In [161]:
#Enter Pub Date between single quotes below

#########################################################################################################################
pubdate = '20200831'
#########################################################################################################################
print (pubdate)

20200831


In [6]:
#Enter Data Release Title between single quotes below

#########################################################################################################################
title = 'Airborne magnetic and radiometric survey over northwest Arkansas, 2019-2020; Radiometric Data'
#########################################################################################################################
print (title)

Airborne magnetic and radiometric survey over northwest Arkansas, 2019-2020; Radiometric Data


In [7]:
#Enter Other Citation between single quotes below

#########################################################################################################################
othercit = 'Additional information about Originators: Anne E. McCafferty, https://orcid.org/0000-0001-5574-9201; Philip J. Brown, http://orcid.org/0000-0002-2415-7462'
#########################################################################################################################
print (othercit)

Additional information about Originators: Anne E. McCafferty, https://orcid.org/0000-0001-5574-9201; Philip J. Brown, http://orcid.org/0000-0002-2415-7462


In [8]:
#Enter Online Link between single quotes below

##########################################################################################################################
onlink = 'https://doi.org/10.5066/P91O2Y8W'
##########################################################################################################################
print (onlink)

https://doi.org/10.5066/P91O2Y8W


### -- Specify purpose and abstract to refer to child data item only --

In [9]:
#Enter Data Release Abstract between single quotes below

###############################################################
abstract = 'This data release child item contains digital flight line data for a high-resolution radiometric survey over an area of northwest Arkansas. The airborne geophysical survey was jointly funded by the USGS National Cooperative Geologic Mapping Program and the Earth Mapping Resource Initiative and was designed to meet the complimentary needs related to geologic mapping and mineral resource research. The survey covers an irregular area 225 by 110 km over the transition between the Ouachita and Ozarks Plateau physiographic provinces and includes 67,614 line km of radiometric data. Data were collected from a fixed-wing aircraft flown at mean terrain clearance of 167 meters (m) above topography along N-S flight lines spaced at 200 m intervals. Tie lines were flown in an E-W direction every 3000 m. Data were collected by EON Geosciences under contract with the USGS using a fixed wing aircraft with a fully calibrated gamma-ray spectrometer. The survey operated out of the Branson, Missouri airport from October of 2019 to March of 2020. Files that are available in this child item include data for the radiometric survey and geotiffs of the gridded results.'
print (abstract)

This data release child item contains digital flight line data for a high-resolution radiometric survey over an area of northwest Arkansas. The airborne geophysical survey was jointly funded by the USGS National Cooperative Geologic Mapping Program and the Earth Mapping Resource Initiative and was designed to meet the complimentary needs related to geologic mapping and mineral resource research. The survey covers an irregular area 225 by 110 km over the transition between the Ouachita and Ozarks Plateau physiographic provinces and includes 67,614 line km of radiometric data. Data were collected from a fixed-wing aircraft flown at mean terrain clearance of 167 meters (m) above topography along N-S flight lines spaced at 200 m intervals. Tie lines were flown in an E-W direction every 3000 m. Data were collected by EON Geosciences under contract with the USGS using a fixed wing aircraft with a fully calibrated gamma-ray spectrometer. The survey operated out of the Branson, Missouri airpor

In [10]:
#Enter Data Release Purpose between single quotes below

#########################################################################################################################
purpose = 'This radiometric survey was flown as a component of U.S. Geological Survey (USGS) projects to improve the geologic framework for research related to mineral resource, energy resource, and geologic hazards studies in the area. Results from the survey will aid in providing insight into the radioelement signatures of exposed and concealed geology.'
#########################################################################################################################
print (purpose)

This radiometric survey was flown as a component of U.S. Geological Survey (USGS) projects to improve the geologic framework for research related to mineral resource, energy resource, and geologic hazards studies in the area. Results from the survey will aid in providing insight into the radioelement signatures of exposed and concealed geology.


## Make sure to add correct file listing below

At some point this could be automated if we stuck to a file directory schema convention

In [11]:
# Enter Data Release Supplementary Information between single quotes below
# U

#########################################################################################################################
supplinf = 'References cited: \n'\
+ '\nDay, W.D., 2019, The Earth MappingResources Initiative (Earth MRI): Mapping the nations critical mineral resources: U.S. Geological Survey Fact Sheet 2019-3007, https://doi.org/10.3133/fs20193007. \n'\
+ '\nThis metadata describes the following files:'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_Raster_EqTotalCounts.png'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqThorium.tif' \
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqPotassium.tif'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqUranium.tif'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqTC.tif'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_UThK-TertiaryImage.tif'\
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricData-RadSpecCulled.csv' \
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricSpecDRawWaveforms.csv' \
+ '\nUSA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricSpecURawWaveforms.csv'
#########################################################################################################################
print (supplinf)

References cited: 

Day, W.D., 2019, The Earth MappingResources Initiative (Earth MRI): Mapping the nations critical mineral resources: U.S. Geological Survey Fact Sheet 2019-3007, https://doi.org/10.3133/fs20193007. 

This metadata describes the following files:
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Raster_EqTotalCounts.png
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqThorium.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqPotassium.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqUranium.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqTC.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_UThK-TertiaryImage.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricData-RadSpecCulled.csv
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricSpecDRawWaveforms.csv
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricSpecURawWaveform

In [12]:
#Enter Data Release Begining Date between single quotes below

#########################################################################################################################
begdate = '20191005'
#########################################################################################################################
print (begdate)

20191005


In [13]:
#Enter Data Release Begining Date between single quotes below

#########################################################################################################################
enddate = '20200307'
#########################################################################################################################
print (enddate)

20200307


In [14]:
#Enter Data Release Access Constraints between single quotes below

#########################################################################################################################
accconst = 'Please see "'"Distribution Info"'" for details.'
#########################################################################################################################
print (accconst)

Please see "Distribution Info" for details.


In [15]:
#Enter Data Release Use Constraints between single quotes below

#########################################################################################################################
useconst = 'There is no guarantee concerning the accuracy of the data. Any user who modifies the data is obligated to describe the types of modifications they perform. Data have been checked to ensure the accuracy. If any errors are detected, please notify the originating office. The U.S. Geological Survey strongly recommends that careful attention be paid to the metadata file associated with these data. Acknowledgment of the U.S. Geological Survey would be appreciated in products derived from these data. User specifically agrees not to misrepresent the data, nor to imply that changes made were approved or endorsed by the U.S. Geological Survey. Please refer to http://www.usgs.gov/privacy.html for the USGS disclaimer.'
#########################################################################################################################
print (useconst)

There is no guarantee concerning the accuracy of the data. Any user who modifies the data is obligated to describe the types of modifications they perform. Data have been checked to ensure the accuracy. If any errors are detected, please notify the originating office. The U.S. Geological Survey strongly recommends that careful attention be paid to the metadata file associated with these data. Acknowledgment of the U.S. Geological Survey would be appreciated in products derived from these data. User specifically agrees not to misrepresent the data, nor to imply that changes made were approved or endorsed by the U.S. Geological Survey. Please refer to http://www.usgs.gov/privacy.html for the USGS disclaimer.


In [16]:
#Enter Data Release Data Contact between single quotes below

#########################################################################################################################
cntperDataContact = 'Anne E. McCafferty'
#########################################################################################################################
print (cntperDataContact)

Anne E. McCafferty


In [17]:
#Enter Data Release Data Contact's Position between single quotes below

#########################################################################################################################
cntposDataContact = 'Research Geophysicist'
#########################################################################################################################
print (cntposDataContact)

Research Geophysicist


In [18]:
#Enter Data Release Data Voice Contact Phone Number between single quotes below

#########################################################################################################################
cntvoiceDataContact = '303-236-1397'
#########################################################################################################################
print (cntvoiceDataContact)

303-236-1397


In [19]:
#Enter Data Release Email Contact between single quotes below

#########################################################################################################################
cntemailDataContact = 'anne@usgs.gov'
#########################################################################################################################
print (cntemailDataContact)

anne@usgs.gov


In [20]:
#Enter Data Release datacred between single quotes below

#########################################################################################################################
datacred = 'This survey and release of the dataset was funded by the Earth Mapping Resources Initiative (Earth MRI).'
#########################################################################################################################
print (datacred)

This survey and release of the dataset was funded by the Earth Mapping Resources Initiative (Earth MRI).


In [21]:
#Enter Data Release attraccr between single quotes below

#########################################################################################################################
attraccr = 'Daily quality control of raw radiometric and magnetic data was performed by the contractor. Further quality control measures occurred post flight by the contractor and USGS scientist-in-charge before approval and acceptance of the final data. The data have been peer reviewed and compared with related ancillary data.'
#########################################################################################################################
print (attraccr)

Daily quality control of raw radiometric and magnetic data was performed by the contractor. Further quality control measures occurred post flight by the contractor and USGS scientist-in-charge before approval and acceptance of the final data. The data have been peer reviewed and compared with related ancillary data.


In [22]:
#Enter Data Release logic between single quotes below

#########################################################################################################################
logic = 'Data were visually inspected for reasonableness and spatial consistency.'
#########################################################################################################################
print (logic)

Data were visually inspected for reasonableness and spatial consistency.


In [23]:
#Enter Data Release complete between single quotes below

#########################################################################################################################
complete = 'Data set is considered complete for the information presented, as described in the abstract. Users are advised to read the rest of the metadata record carefully for additional details.'
#########################################################################################################################
print (complete)

Data set is considered complete for the information presented, as described in the abstract. Users are advised to read the rest of the metadata record carefully for additional details.


In [24]:
#Enter Data Release horizpar between single quotes below

#########################################################################################################################
horizpar = 'Field quality control and assurance procedures were undertaken by the Eon Geosciences Inc. field crew and are described in the contractors report included with this data release.'
#########################################################################################################################
print (horizpar)

Field quality control and assurance procedures were undertaken by the Eon Geosciences Inc. field crew and are described in the contractors report included with this data release.


In [25]:
#Enter Data Release vertaccr between single quotes below

#########################################################################################################################
vertaccr = 'Field quality control and assurance procedures were undertaken by the Eon Geosciences Inc. field crew and are described in the contractors report included with this data release.'
#########################################################################################################################
print (vertaccr)

Field quality control and assurance procedures were undertaken by the Eon Geosciences Inc. field crew and are described in the contractors report included with this data release.


In [26]:
#Enter Data Release latres between single quotes below

#########################################################################################################################
latres = '0.000001'
#########################################################################################################################
print (latres)

0.000001


In [27]:
#Enter Data Release Data longres between single quotes below

#########################################################################################################################
longres = '0.000001'
#########################################################################################################################
print (longres)

0.000001


In [28]:
#Enter Data Release horizdn between single quotes below

#########################################################################################################################
horizdn = 'WGS_1984'
#########################################################################################################################
print (horizdn)


WGS_1984


In [29]:
#Enter Data Release ellips between single quotes below

#########################################################################################################################
ellips = 'WGS_1984'
#########################################################################################################################
print (ellips)

WGS_1984


In [30]:
#Enter Data Release semiaxis between single quotes below

#########################################################################################################################
semiaxis = '6378206.4'
#########################################################################################################################
print (semiaxis)


6378206.4


In [31]:
#Enter Data Release denflat between single quotes below

#########################################################################################################################
denflat = '294.9786982'
#########################################################################################################################
print (denflat)

294.9786982


In [32]:
#Enter Data Release Data Contac's Phone Number between single quotes below

#########################################################################################################################
distliab = 'Unless otherwise stated, all data, metadata and related materials are considered to satisfy the quality standards relative to the purpose for which the data were collected. Although these data and associated metadata have been reviewed for accuracy and completeness and approved for release by the U.S. Geological Survey (USGS), no warranty expressed or implied is made regarding the display or utility of the data on any other system or for general or scientific purposes, nor shall the act of distribution constitute any such warranty.'
#########################################################################################################################
print (distliab)

Unless otherwise stated, all data, metadata and related materials are considered to satisfy the quality standards relative to the purpose for which the data were collected. Although these data and associated metadata have been reviewed for accuracy and completeness and approved for release by the U.S. Geological Survey (USGS), no warranty expressed or implied is made regarding the display or utility of the data on any other system or for general or scientific purposes, nor shall the act of distribution constitute any such warranty.


In [33]:
#Enter Data Release metadata creation date between single quotes below

#########################################################################################################################
metd = '20200708'
#########################################################################################################################
print (metd)

20200708


In [34]:
#Enter Data Release cntperMetaInfo between single quotes below

#########################################################################################################################
cntperMetaInfo = 'Philip Brown'
#########################################################################################################################
print (cntperMetaInfo)

Philip Brown


In [35]:
#Enter Data Release cntposMetaInfo between single quotes below

#########################################################################################################################
cntposMetaInfo = 'Geophysicist'
#########################################################################################################################
print (cntposMetaInfo)

Geophysicist


In [36]:
#Enter Data Release cntvoiceMetaInfo between single quotes below

#########################################################################################################################
cntvoiceMetaInfo = '303-236-1310'
#########################################################################################################################
print (cntvoiceMetaInfo)

303-236-1310


In [37]:
#Enter Data Release cntemailMetaInfo between single quotes below

#########################################################################################################################
cntemailMetaInfo = 'pbrown@usgs.gov'
#########################################################################################################################
print (cntemailMetaInfo)

pbrown@usgs.gov


In [38]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrThemeISO = ['geoscientificInformation','economy']
#########################################################################################################################

txtThemeISO = '\t\t\t<theme>\n\t\t\t\t<themekt>ISO 19115 Topic Category</themekt>'
for i in range(len(arrThemeISO)):
 txtThemeISO = txtThemeISO +  '\n\t\t\t\t<themekey>' + arrThemeISO[i] + '</themekey>'

txtThemeISO = txtThemeISO + '\n\t\t\t</theme>'
print(str(txtThemeISO))


			<theme>
				<themekt>ISO 19115 Topic Category</themekt>
				<themekey>geoscientificInformation</themekey>
				<themekey>economy</themekey>
			</theme>


In [39]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrThemeNONE = ['Mineral Resources Program','MRP', 'Geology, Geophysics, and Geochemistry Science Center' \
            'GGGSC','Earth Mapping Resources Initiative','EarthMRI','horizontal magnetic gradiometry', \
            'vertical magnetic gradient','airborne geophysical survey','U.S. Geological Survey','USGS', \
            'Association of American State Geologists','AASG','EON Geosciences, Inc.','radon','Rn']
#########################################################################################################################

txtThemeNONE = '\n\t\t\t<theme>\n\t\t\t\t<themekt>None</themekt>'
for i in range(len(arrThemeNONE)):
 txtThemeNONE = txtThemeNONE +  '\n\t\t\t\t<themekey>' + arrThemeNONE[i] + '</themekey>'

txtThemeNONE = txtThemeNONE + '\n\t\t\t</theme>'
print(str(txtThemeNONE))


			<theme>
				<themekt>None</themekt>
				<themekey>Mineral Resources Program</themekey>
				<themekey>MRP</themekey>
				<themekey>Geology, Geophysics, and Geochemistry Science CenterGGGSC</themekey>
				<themekey>Earth Mapping Resources Initiative</themekey>
				<themekey>EarthMRI</themekey>
				<themekey>horizontal magnetic gradiometry</themekey>
				<themekey>vertical magnetic gradient</themekey>
				<themekey>airborne geophysical survey</themekey>
				<themekey>U.S. Geological Survey</themekey>
				<themekey>USGS</themekey>
				<themekey>Association of American State Geologists</themekey>
				<themekey>AASG</themekey>
				<themekey>EON Geosciences, Inc.</themekey>
				<themekey>radon</themekey>
				<themekey>Rn</themekey>
			</theme>


In [40]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrThemeUSGS = ['magnetic field (earth)','geophysics', 'geospatial datasets', \
            'aeromagnetic surveying','aeroradiometric surveying','potassium','uranium', \
            'thorium']
#########################################################################################################################

txtThemeUSGS = '\n\t\t\t<theme>\n\t\t\t\t<themekt>USGS Thesaurus</themekt>'
for i in range(len(arrThemeUSGS)):
 txtThemeUSGS = txtThemeUSGS +  '\n\t\t\t\t<themekey>' + arrThemeUSGS[i] + '</themekey>'

txtThemeUSGS = txtThemeUSGS + '\n\t\t\t</theme>'
print(str(txtThemeUSGS))


			<theme>
				<themekt>USGS Thesaurus</themekt>
				<themekey>magnetic field (earth)</themekey>
				<themekey>geophysics</themekey>
				<themekey>geospatial datasets</themekey>
				<themekey>aeromagnetic surveying</themekey>
				<themekey>aeroradiometric surveying</themekey>
				<themekey>potassium</themekey>
				<themekey>uranium</themekey>
				<themekey>thorium</themekey>
			</theme>


In [41]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrPlaceGNIS = ['Boston Mountains','Carroll County', 'Madison County', \
            'Franklin County','Johnson County','Pope County','Newton County', \
            'Boone County','Marion County','Baxter County', 'Stone County', \
            'Izard County','Sharp County','Independence County','Lawrence County', \
            'Searcy County','Washington County']
#########################################################################################################################

txtPlaceGNIS = '\n\t\t\t<place>\n\t\t\t\t<placekt>USGS Geographic Names Information System (GNIS), https://geonames.usgs.gov </placekt>'
for i in range(len(arrPlaceGNIS)):
 txtPlaceGNIS = txtPlaceGNIS +  '\n\t\t\t\t<placekey>' + arrPlaceGNIS[i] + '</placekey>'


txtPlaceGNIS = txtPlaceGNIS + '\n\t\t\t</place>'
print(str(txtPlaceGNIS))


			<place>
				<placekt>USGS Geographic Names Information System (GNIS), https://geonames.usgs.gov </placekt>
				<placekey>Boston Mountains</placekey>
				<placekey>Carroll County</placekey>
				<placekey>Madison County</placekey>
				<placekey>Franklin County</placekey>
				<placekey>Johnson County</placekey>
				<placekey>Pope County</placekey>
				<placekey>Newton County</placekey>
				<placekey>Boone County</placekey>
				<placekey>Marion County</placekey>
				<placekey>Baxter County</placekey>
				<placekey>Stone County</placekey>
				<placekey>Izard County</placekey>
				<placekey>Sharp County</placekey>
				<placekey>Independence County</placekey>
				<placekey>Lawrence County</placekey>
				<placekey>Searcy County</placekey>
				<placekey>Washington County</placekey>
			</place>


In [42]:
#Let's create a key words array for each dictionary type - then loop through arrays and concatenate to {keywords}
#Add text below in single quotes seperated by commas

#########################################################################################################################
arrPlaceGNIS = ['Ouachita Plateau','Ozarks Plateau', 'Arkansas', \
            'United States of America','USA']
#########################################################################################################################

txtPlaceGNIS = '\n\t\t\t<place>\n\t\t\t\t<placekt>Common geographic areas</placekt>'
for i in range(len(arrPlaceGNIS)):
 txtPlaceGNIS = txtPlaceGNIS +  '\n\t\t\t\t<placekey>' + arrPlaceGNIS[i] + '</placekey>'

txtPlaceGNIS = txtPlaceGNIS + '\n\t\t\t</place>'
print(str(txtPlaceGNIS))


			<place>
				<placekt>Common geographic areas</placekt>
				<placekey>Ouachita Plateau</placekey>
				<placekey>Ozarks Plateau</placekey>
				<placekey>Arkansas</placekey>
				<placekey>United States of America</placekey>
				<placekey>USA</placekey>
			</place>


In [118]:
## Create editable keywords example.  
## Example text is created after running this cell
## This text is displayed by running "display(keywords) below
keywords = '\t\t<keywords>\n' + txtThemeISO + txtThemeNONE + txtThemeUSGS + txtPlaceGNIS \
 + txtPlaceGNIS + '\n\t\t</keywords>'
print (keywords)

		<keywords>
			<theme>
				<themekt>ISO 19115 Topic Category</themekt>
				<themekey>geoscientificInformation</themekey>
				<themekey>economy</themekey>
			</theme>
			<theme>
				<themekt>None</themekt>
				<themekey>Mineral Resources Program</themekey>
				<themekey>MRP</themekey>
				<themekey>Geology, Geophysics, and Geochemistry Science CenterGGGSC</themekey>
				<themekey>Earth Mapping Resources Initiative</themekey>
				<themekey>EarthMRI</themekey>
				<themekey>horizontal magnetic gradiometry</themekey>
				<themekey>vertical magnetic gradient</themekey>
				<themekey>airborne geophysical survey</themekey>
				<themekey>U.S. Geological Survey</themekey>
				<themekey>USGS</themekey>
				<themekey>Association of American State Geologists</themekey>
				<themekey>AASG</themekey>
				<themekey>EON Geosciences, Inc.</themekey>
				<themekey>radon</themekey>
				<themekey>Rn</themekey>
			</theme>
			<theme>
				<themekt>USGS Thesaurus</themekt>
				<themekey>magnetic field (eart

# Create process descriptions and dates

In [139]:
#Now let's create the lineage 2-D array of Process steps and dates
#Add text below in single quotes seperated by commas.  Sequencial 2-D arrays are seperated by brackets []
#2-D Array form is [['procdesc1','procdate1'],['procdesc2','procdate2'],etc...]

######################################################################################################################################################################################################################################################################################################################################################################################
arrLineage = [ \
             ['The airborne survey was flown under contract to the U.S. Geological Survey by EON Geosciences Inc., from October 5, 2019 to March 7, 2020. The magnetic data were acquired with a Geometrics G-822A cesium-vapour split-beam sensor. The radiometric data were acquired with a Radiation Solutions RS-500 series multi-channel gamma-ray spectrometers, models RSX-5 and RSX-4, with 1,024 channels, for a total volume of 33.6 liters downward-looking and 8.4 liters (C-FEON) or 4.2 liters (C-GPTB) upward-looking crystals (for measuring effects of atmospheric radon). Data were collected along east-west flight lines spaced 200 meters (m) apart with north-south tie lines flown every 3,000 meters. A mean terrain clearance of 80 m was maintained except where safety dictated a higher elevation. A total of 66,477 line-kilometers of data were collected.', '20191005'] \
             ,['Preliminary magnetic and radiometric data processing was undertaken by EON Geosciences, Inc. of Montreal, Quebec, Canada. This processing workflow is described in the contractors report included with this data release.', '20200316'] \
             ,['Final data and contractors report were accepted and received by USGS from EON Geosciences, Inc.','20200629'] \
             ,['Georeferenced raster images of the data grids provided by the contractor as well as Esri shapefiles of area footprint and line paths for the survey were developed from the radiometric and magnetic data.','20200707'] \
             ,['Final data were exported to CSV-format to meet open data requirements.','20200707']
             ]
######################################################################################################################################################################################################################################################################################################################################################################################

txtLineage = '\t\t<lineage>'
for i in range(len(arrLineage)):
 txtLineage = txtLineage +  '\n\t\t\t<procstep>\n\t\t\t\t<procdesc>' + arrLineage[i][0] + '</procdesc>\n\t\t\t\t' \
    + '<procdate>' + arrLineage[i][1] + '</procdate>\n\t\t\t</procstep>'

txtLineage = txtLineage + '\n\t\t</lineage>'
print(str(txtLineage))
lineage = txtLineage

		<lineage>
			<procstep>
				<procdesc>The airborne survey was flown under contract to the U.S. Geological Survey by EON Geosciences Inc., from October 5, 2019 to March 7, 2020. The magnetic data were acquired with a Geometrics G-822A cesium-vapour split-beam sensor. The radiometric data were acquired with a Radiation Solutions RS-500 series multi-channel gamma-ray spectrometers, models RSX-5 and RSX-4, with 1,024 channels, for a total volume of 33.6 liters downward-looking and 8.4 liters (C-FEON) or 4.2 liters (C-GPTB) upward-looking crystals (for measuring effects of atmospheric radon). Data were collected along east-west flight lines spaced 200 meters (m) apart with north-south tie lines flown every 3,000 meters. A mean terrain clearance of 80 m was maintained except where safety dictated a higher elevation. A total of 66,477 line-kilometers of data were collected.</procdesc>
				<procdate>20191005</procdate>
			</procstep>
			<procstep>
				<procdesc>Preliminary magnetic and radio

# Set Directory Paths
## Please set directory paths below
### Directory paths include
- Data Path: This is the path to the data, data structure should have a directory for each station
- Template Path: The path to the XML metadata template being used for the data.  This template should already include all information common to all child metadata files e.g. originators, larger work citation, etc.

In [174]:
#Set Data Paths - perhaps we'll get a user form to do this some day?
# The r signifies a string literal. Use for paths.

radMetaDataTemplatePath = r"C:\CurrentWork\DataReleases\NorthwesternArkansas\Metadata"
radMetaDataTemplateName = "USA_Northwest-Arkansas-Airborne-Survey-2019-2020--TEMPLATE.xml"
radCSVfileDir = r"C:\CurrentWork\DataReleases\NorthwesternArkansas\CSV_Data"
radCSVfileName = "USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricData-RadSpecCulled.csv"

In [141]:
#Check Paths for the fun of it
radCSVfilePath = radCSVfileDir + '\\' + radCSVfileName
print ('The Rad CSV Data Path is: ' + '"' + radCSVfilePath + '"')#magCSVfilePath
radMetadataTemplatefilePath = radMetaDataTemplatePath + "\\" + radMetaDataTemplateName
print ('The Template File Path is: ' + '"' + radMetadataTemplatefilePath + '"')

The Rad CSV Data Path is: "C:\CurrentWork\DataReleases\NorthwesternArkansas\CSV_Data\USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricData-RadSpecCulled.csv"
The Template File Path is: "C:\CurrentWork\DataReleases\NorthwesternArkansas\Metadata\USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Magnetic--TEMPLATE.xml"


In [142]:
tifFilesPath = r"C:\CurrentWork\DataReleases\NorthwesternArkansas\Geotiffs\Rad"
tifList = glob.glob(os.path.join(tifFilesPath, '*.tif'),  recursive=True)
fileListing = ''
rspFileListing = []
for i in range(len(tifList)):
    splitTifList = tifList[i].split('\\')
    fileListing = fileListing +  splitTifList[len(splitTifList) - 1] + '\n'
    rspFileListing.append(splitTifList[len(splitTifList) - 1])
print (fileListing)

USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqPotassium.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqTC.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqThorium.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqUranium.tif
USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_UThK-TertiaryImage.tif



## Lets now import and index values from the CSV file
- We need create arrays of description and unit values for the metadata template.  
- We also want to run stats on the csv columns for the entity and attributes section

In [48]:
# read file into pandas; note that can take over 1 minute depending on the size of the CSV file and computing power.
print ('!!! PLEASE BE PATIENT AND TAKE SEVERAL BREATHS !!!')
print ('Reading file into Pandas, this may take several minutes depending on the file size...')

radDataFrame = pd.read_csv(radCSVfilePath, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
radDataFrame

!!! PLEASE BE PATIENT AND TAKE SEVERAL BREATHS !!!
Reading file into Pandas, this may take several minutes depending on the file size...


,attK_c,attK_o,attTC_c,attTC_o,attTH_c,attTH_o,attU_c,attU_o,baltl,baltlc,...,THraw,topoSRTM,UKratio,upUraw,Uraw,UTHratio,x,y,year,z
0,29.857597,29.857597,645.391602,645.391602,29.238268,29.238268,6.147030,6.147030,434.727020,434.463745,...,38,457.549774,2.061743,3,30,0.143437,426297.15625,4012245.75,2019,568.612793
1,31.166235,31.166235,644.757996,644.757996,26.777546,26.777546,9.429593,9.429593,433.766449,433.714111,...,29,450.987427,2.308758,7,38,0.169075,426295.84375,4012170.50,2019,567.800476
2,32.348354,32.348354,652.919067,652.919067,24.696476,24.696476,12.373293,12.373293,432.523926,432.731476,...,34,450.662781,3.197426,7,40,0.255764,426295.18750,4012095.00,2019,566.784119
3,32.081871,32.081871,670.677307,670.677307,23.329210,23.329210,14.769251,14.769251,431.544861,431.415955,...,28,448.636139,3.620701,2,31,0.297423,426294.56250,4012019.75,2019,565.460815
4,30.803078,30.803078,683.499084,683.499084,22.553476,22.553476,16.614653,16.614653,429.681885,429.720154,...,39,450.610413,4.433259,8,34,0.380961,426294.59375,4011944.50,2019,563.682251
5,30.500582,30.500582,678.609497,678.609497,21.989862,21.989862,17.785076,17.785076,428.120331,427.653564,...,29,451.327209,4.636347,5,41,0.401648,426293.96875,4011868.25,2019,561.462830
6,32.210697,32.210697,665.907471,665.907471,21.401140,21.401140,17.865896,17.865896,424.706696,425.240967,...,32,449.700195,4.350427,5,41,0.397174,426294.00000,4011792.50,2019,558.651184
7,34.531555,34.531555,663.675110,663.675110,20.908918,20.908918,16.393038,16.393038,421.896271,422.491821,...,26,446.731598,4.003246,7,33,0.371398,426294.03125,4011716.00,2019,556.198364
8,35.419903,35.419903,668.276306,668.276306,20.831076,20.831076,13.357880,13.357880,419.980347,419.431213,...,39,441.782806,2.976108,3,42,0.280874,426294.09375,4011639.25,2019,553.916992
9,34.668789,34.668789,662.571899,662.571899,21.289419,21.289419,9.474638,9.474638,416.552429,416.175110,...,19,441.004822,2.500003,7,30,0.232006,426294.12500,4011562.75,2019,550.936768


In [143]:
# now get an array of the header names
radHeaderList = list(radDataFrame.columns.values)
radHeaderString = ''
for i in range(len(radHeaderList)):
 radHeaderString = radHeaderString + radHeaderList[i] + ','

print (radHeaderString)

attK_c,attK_o,attTC_c,attTC_o,attTH_c,attTH_o,attU_c,attU_o,baltl,baltlc,barom,conK_c,conK_d,conK_F,conK_o,conTC_c,conTC_d,conTC_F,conTC_o,conTH_c,conTH_d,conTH_F,conTH_o,conU_c,conU_d,conU_F,conU_o,COSraw,date,directn,drape,DTMc,DTMo,fid01,fiducial,flt,hgps,Hstp,jul_day,Kraw,lat,latitude,line,line_no,lon,longitud,lt,ndd,nud,Preslc,radar,raltl01,raltlc01,TCraw,Templc,THKratio,THraw,topoSRTM,UKratio,upUraw,Uraw,UTHratio,x,y,year,z,


In [144]:
# Get Min and Max Latitude and Longitude
#---------------------------------------------------------------------------------
#*************!!!MAKE SURE LAT AND LONG CHANNELS ARE CORRECT BELOW!!!*************
#********* ENTER CORRECT LAT/LONG CHANNEL NAMES BETWEEN SINGLE QUOTES ************
#*****!!!Make sure the results are rounded to the correct # of Sig Figs!!!********
#---------------------------------------------------------------------------------

########################################################################################################################
westbc = radDataFrame['longitud'].dropna().max()
########################################################################################################################
westbc = float(westbc)    # convert to float so that number can be rounded 
westbc = round(westbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

#########################################################################################################################
eastbc = radDataFrame['longitud'].dropna().min()
#########################################################################################################################
eastbc = float(eastbc)    # convert to float so that number can be rounded  
eastbc = round(eastbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

#########################################################################################################################
northbc = radDataFrame['latitude'].dropna().max()
#########################################################################################################################
northbc = float(northbc)    # convert to float so that number can be rounded 
northbc = round(northbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

#########################################################################################################################
southbc = radDataFrame['latitude'].dropna().min()
#########################################################################################################################
southbc = float(southbc)    # convert to float so that number can be rounded  
southbc = round(southbc, 6) # round number to indicated number of sig figs
#=======================================================================================================================

print ('westbc: ' + str(westbc))
print ('eastbc: ' + str(eastbc))
print ('northbc: ' + str(northbc))
print ('southbc: ' + str(southbc))

westbc: -91.323524
eastbc: -93.820724
northbc: 36.500267
southbc: 35.51334


### EandA Channel Templates for the magnetic data are available at the following link to a shared Jupyter notebook.  

### -- !!! Be sure to have the correct entity and attributes listed for your data set !!! --

https://github.com/pbrown-usgs/SquamataAeroMag/blob/master/SquamataAeroRadEntityAndAttributeArchive.ipynb

In [162]:
#Create Entity and Attribute Text - substitute vales below with definitions from contractors report:
# EandA Channel Templates for the radiometric data are available at the following link to a shared Jupyter Notebook
# https://github.com/pbrown-usgs/SquamataAeroMag/blob/master/SquamataAeroRadEntityAndAttributeArchive.ipynb

#Create Entity and Attribute Text FOR RAD:
csvAttributes = ''
csvEntityDescription = []
csvEntityUnit = []
csvMax = []
csvMin = []
#=======================================================================================================================
# Add entity type and detailed tags
# Below is the entity type description for the Down-Raw spectrum waveforms
entAttText = '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>' + radCSVfileName + '</enttypl>\n' \
  + '\t\t\t\t<enttypd>Table containing attribute information associated with the data set.</enttypd>' \
  + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>'

##-----------------Start EandA here
csvEntityDescription.append("Potassium count, attenuated. Limit high-frequency over-attenuation >265m STP on [attK_o].")#attK_c
csvEntityUnit.append("cps")#attK_c
#=======================================================================================================================
csvEntityDescription.append("Potassium count, attenuated. Attenuation to 100m, final calibrations used.")#attK_o
csvEntityUnit.append("cps")#attK_o
#=======================================================================================================================
csvEntityDescription.append("Total count, attenuated. Limit high-frequency over-attenuation >265m STP on [attTC_o].")#attTC_c
csvEntityUnit.append("cps")#attTC_c
#=======================================================================================================================
csvEntityDescription.append("Air-absorbed dose rate. Computed using sensitivity at 100m from [attTC_o].")#attTC_o
csvEntityUnit.append("nGy/h")#attTC_o
#=======================================================================================================================
csvEntityDescription.append("Thorium count, attenuated.  Limit high-frequency over-attenuation >265m STP on [attTH_o].")#attTH_c
csvEntityUnit.append("cps")#attTH_c
#=======================================================================================================================
csvEntityDescription.append("Thorium eq. concentration. Computed using sensitivity at 100m from [attTH_o].")#attTH_o
csvEntityUnit.append("ppm")#attTH_o
#=======================================================================================================================
csvEntityDescription.append("Uranium count, attenuated. Limit high-frequency over-attenuation >265m STP on [attU_o].")#attU_c
csvEntityUnit.append("cps")#attU_c
#=======================================================================================================================
csvEntityDescription.append("Uranium count, attenuated. Attenuation to 100m, final calibrations used")#attU_o
csvEntityUnit.append("cps")#attU_o
#=======================================================================================================================
csvEntityDescription.append("Baro altitude, raw. MSL, computed from measured pressure and temperature")#baltl
csvEntityUnit.append("meters")#baltl
#=======================================================================================================================
csvEntityDescription.append("Baro altitude, edited. MSL, low-pass filtered against noise and spikes")#baltlc
csvEntityUnit.append("meters")#baltlc
#=======================================================================================================================
csvEntityDescription.append("Barometric altitude above MSL. Same as [baltlc].")#barom
csvEntityUnit.append("meters")#barom
#=======================================================================================================================
csvEntityDescription.append("Potassium concentration.  Computed using sensitivity at 100m from[attK_c].")#conK_c
csvEntityUnit.append("percent")#conK_c
#=======================================================================================================================
csvEntityDescription.append("Potassium concentration.  Directional filtering applied to [conK_c].")#conK_d
csvEntityUnit.append("percent")#conK_d
#=======================================================================================================================
csvEntityDescription.append("Potassium concentration. Non-directional grid-based filtering applied to [conK_d]")#conK_F
csvEntityUnit.append("percent")#conK_F
#=======================================================================================================================
csvEntityDescription.append("Potassium concentration. Computed using sensitivity at 100m from [attK_o]")#conK_o
csvEntityUnit.append("percent")#conK_o
#=======================================================================================================================
csvEntityDescription.append("Air-absorbed dose rate. Computed using sensitivity at 100m from[attTC_c].")#conTC_c
csvEntityUnit.append("nGy/h")#conTC_c
#=======================================================================================================================
csvEntityDescription.append("Air-absorbed dose rate. Directional filtering applied to[conTC_c].")#conTC_d
csvEntityUnit.append("nGy/h")#conTC_d
#=======================================================================================================================
csvEntityDescription.append("Air-absorbed dose rate. Non-directional grid-based filtering applied to [conTC_d].")#conTC_F
csvEntityUnit.append("nGy/h")#conTC_F
#=======================================================================================================================
csvEntityDescription.append("Air-absorbed dose rate. Computed using sensitivity at 100m from [attTC_o].")#conTC_o
csvEntityUnit.append("nGy/h")#conTC_o
#=======================================================================================================================
csvEntityDescription.append("Thorium eq. concentration. Computed using sensitivity at 100m from [attTH_c].")#conTH_c
csvEntityUnit.append("nGy/h")#conTH_c
#=======================================================================================================================
csvEntityDescription.append("Thorium eq. concentration. Directional filtering applied to[conTH_c].")#conTH_d
csvEntityUnit.append("ppm")#conTH_d
#=======================================================================================================================
csvEntityDescription.append("Air-absorbed dose rate. Non-directional grid-based filtering applied to [conTC_d].")#conTH_F
csvEntityUnit.append("nGy/h")#conTH_F
#=======================================================================================================================
csvEntityDescription.append("Thorium eq. concentration. Computed using sensitivity at 100m from [attTH_o].")#conTH_o
csvEntityUnit.append("ppm")#conTH_o
#=======================================================================================================================
csvEntityDescription.append("Uranium eq. concentration. Computed using sensitivity at 100m from [attU_c]")#conU_c
csvEntityUnit.append("ppm")#conU_c
#=======================================================================================================================
csvEntityDescription.append("Uranium eq. concentration. Directional filtering applied to [conU_c].")#conU_d
csvEntityUnit.append("ppm")#conU_d
#=======================================================================================================================
csvEntityDescription.append("Uranium eq. concentration. Non-directional grid-based filtering applied to [conU_d].")#conU_F
csvEntityUnit.append("ppm")#conU_F
#=======================================================================================================================
csvEntityDescription.append("Uranium eq. concentration. Computed using sensitivity at 100m from [attU_o].")#conU_o
csvEntityUnit.append("ppm")#conU_o
#=======================================================================================================================
csvEntityDescription.append("Cosmic count, raw; 33.6L DOWN, window >3000 keV.")#COSraw
csvEntityUnit.append("cps")#COSraw
#=======================================================================================================================
csvEntityDescription.append("Acquisition date; UTC, start of acquisition.")#date
csvEntityUnit.append("yyyy/mm/dd")#date
#=======================================================================================================================
csvEntityDescription.append("Flight line direction. Azimuth degrees from north.")#directn
csvEntityUnit.append("degrees")#directn
#=======================================================================================================================
csvEntityDescription.append("Drape altitude; MSL, planned flight surface.")#drape
csvEntityUnit.append("meters")#drape
#=======================================================================================================================
csvEntityDescription.append("Digital terrain model. MSL, DTM model, [z-2.5m-raltlc01], GPS ant. 2.5m above radar ant.")#DTMc
csvEntityUnit.append("meters")#DTMc
#=======================================================================================================================
csvEntityDescription.append("Digital terrain model. MSL, DTM model, [z-2.5m-raltl01], GPS ant. 2.5m above radar ant.")#DTMo
csvEntityUnit.append("meters")#DTMo
#=======================================================================================================================
csvEntityDescription.append("Fiducial; Montaj/RMS system fiducial.")#fid01
csvEntityUnit.append("sec UTC")#fid01
#=======================================================================================================================
csvEntityDescription.append("Fiducial number. Same as [fid01] (01Hz version).")#fiducial
csvEntityUnit.append("sec")#fiducial
#=======================================================================================================================
csvEntityDescription.append("Flight number")#flt
csvEntityUnit.append("none")#flt
#=======================================================================================================================
csvEntityDescription.append("GPS time Format HH:MM:SS.SS, synchronized to RMS system")#hgps
csvEntityUnit.append("hour UTC")#hgps
#=======================================================================================================================
csvEntityDescription.append("Corrected TMF value; Same as [mreslvld]")#Hstp
csvEntityUnit.append("nanoteslas")#Hstp
#=======================================================================================================================
csvEntityDescription.append("Julian day flown")#jul_day
csvEntityUnit.append("ddd")#jul_day
#=======================================================================================================================
csvEntityDescription.append("Potassium count, raw. 33.6L DOWN, ch. 0457-0523/1024, window 1370-1570 keV.")#Kraw
csvEntityUnit.append("cps")#Kraw
#=======================================================================================================================
csvEntityDescription.append("Latitude; WGS-84, differential PP-DGPS.")#lat
csvEntityUnit.append("decimal degrees")#lat
#=======================================================================================================================
csvEntityDescription.append("Latitude; Decimal degrees, same as [lat]")#latitude
csvEntityUnit.append("decimal degrees")#latitude
#=======================================================================================================================
csvEntityDescription.append("Line number lll: planned line number, n: segment code")#line
csvEntityUnit.append("llln")#line
#=======================================================================================================================
csvEntityDescription.append("Flight line number. Same as [line].")#line_no
csvEntityUnit.append("llln")#line_no
#=======================================================================================================================
csvEntityDescription.append("Longitude; decimal degrees.")#lon
csvEntityUnit.append("decimal degrees")#lon
#=======================================================================================================================
csvEntityDescription.append("Longitude; Decimal degrees, same as [lon].")#longitud
csvEntityUnit.append("decimal degrees")#longitud
#=======================================================================================================================
csvEntityDescription.append("Live time")#lt
csvEntityUnit.append("msec")#lt
#=======================================================================================================================
csvEntityDescription.append("Number of crystals down")#ndd
csvEntityUnit.append("none")#ndd
#=======================================================================================================================
csvEntityDescription.append("Number of crystals up")#nud
csvEntityUnit.append("none")#nud
#=======================================================================================================================
csvEntityDescription.append("Pressure, edited. Measured pressure, edited against spikes and jumps.")#Preslc
csvEntityUnit.append("mbar")#Preslc
#=======================================================================================================================
csvEntityDescription.append("Radar altitude above ground. Same as [raltlc01] 01Hz version")#radar
csvEntityUnit.append("meters")#radar
#=======================================================================================================================
csvEntityDescription.append("Radar altitude, raw AGL, linear transformation from raw readings (1Hz version)")#raltl01
csvEntityUnit.append("meters")#raltl01
#=======================================================================================================================
csvEntityDescription.append("Radar altitude, edited. AGL, low-pass filtered against noise, spikes and jumps (1Hz version).")#raltlc01
csvEntityUnit.append("meters")#raltLCl01
#=======================================================================================================================
csvEntityDescription.append("cps Total count, raw 33.6L DOWN, ch. 0137-0937/1024, window 0410-2810 keV")#TCraw
csvEntityUnit.append("cps")#TCraw
#=======================================================================================================================
csvEntityDescription.append("Temperature, edited. Measured temperature, edited against spikes and jumps.")#Templc
csvEntityUnit.append("degrees centigrade")#Templc
#=======================================================================================================================
csvEntityDescription.append("Uranium/thorium ratio [conU_f/conTH_f], canceled for aberrant values.")#THKratio
csvEntityUnit.append("ppm/ppm")#THKratio
#=======================================================================================================================
csvEntityDescription.append("Thorium count, raw. 33.6L DOWN, ch. 0803-0937/1024, window 2410-2810 keV")#THraw
csvEntityUnit.append("cps")#THraw
#=======================================================================================================================
csvEntityDescription.append("Published terrain model MSL, DTM model, Shuttle Radar Topography Model.")#topoSRTM
csvEntityUnit.append("meters")#topoSRTM
#=======================================================================================================================
csvEntityDescription.append("Uranium/potassium ratio [conU_f/conK_f], canceled for aberrant values.")#UKratio
csvEntityUnit.append("ppm/percent")#UKratio
#=======================================================================================================================
csvEntityDescription.append("UP-uranium count, raw. 8.4L(EON)/4.2L(PTB) UP, ch. 0553-0620/1024, window 1660-1860 keV")#upUraw
csvEntityUnit.append("cps")#upUraw
#=======================================================================================================================
csvEntityDescription.append("Uranium count, raw. 33.6L DOWN, ch. 0553-0620/1024, window 1660-1860 keV.")#Uraw
csvEntityUnit.append("cps")#Uraw
#=======================================================================================================================
csvEntityDescription.append("Uranium/thorium ratio [conU_f/conTH_f], canceled for aberrant values")#UTHratio
csvEntityUnit.append("ppm/ppm")#UTHratio
#=======================================================================================================================
csvEntityDescription.append("UTM Easting. WGS-84, Z15N, differential PP-DGPS.")#x
csvEntityUnit.append("meters")#x
#=======================================================================================================================
csvEntityDescription.append("UTM Northing. WGS-84, Z15N, differential PP-DGPS.")#y
csvEntityUnit.append("meters")#y
#=======================================================================================================================
csvEntityDescription.append("Year flown.")#year
csvEntityUnit.append("yyyy")#year
#=======================================================================================================================
csvEntityDescription.append("GPS altitude. MSL, differential PP-DGPS")#z
csvEntityUnit.append("meters")#z
#=======================================================================================================================
##-----------------End EandA here

#Now get a min and max value for all columns

csvHeaderListLength = len(csvEntityDescription)
#print ('Length of Entity Description Array: ' + str(len(csvEntityDescription)))
#print ('Length of Header List: ' + str(len(magHeaderList)))

for i in range(csvHeaderListLength): #csvHeaderListLength
 print (radHeaderList[i] + ': ' + str(i))
 strCol = radHeaderList[i]
 csvMax.append(radDataFrame[strCol].max())
 csvMin.append(radDataFrame[strCol].min())
 print ('Max. Value ' + radHeaderList[i] + ' :' + str(csvMax[i]))
 print ('Min. Value ' + radHeaderList[i] + ' :' + str(csvMin[i]))
 print ('Entt Desc. ' + csvEntityDescription[i])
 print ('Att Unit ' + csvEntityUnit[i] + '\n')
 entAtti = '\n\t\t\t\t<attr>\n\t\t\t\t\t<attrlabl>' + strCol + '</attrlabl>\n' \
        + '\t\t\t\t\t<attrdef>' + csvEntityDescription[i] + '</attrdef>' \
        + '\n\t\t\t\t\t<attrdefs>EON GEOSCIENCES INC.</attrdefs>' \
        + '\n\t\t\t\t\t<attrdomv>\n\t\t\t\t\t\t<rdom>\n\t\t\t\t\t\t\t<rdommin>'+ str(csvMin[i]) + '</rdommin>\n\t\t\t\t\t\t\t<rdommax>' \
        + str(csvMax[i]) + '</rdommax>\n\t\t\t\t\t\t\t' + '<attrunit>' + csvEntityUnit[i] + '</attrunit>\n\t\t\t\t\t\t</rdom>\n\t\t\t\t\t\t' \
        + '</attrdomv>\n' \
        + '\t\t\t\t</attr>'
 entAttText = entAttText + entAtti
    
entAttText = entAttText  + '\n\t\t</detailed>\n' 

csvAttributes = entAttText

print ('\nEntity and Attribute Text:\n' + csvAttributes)

attK_c: 0
Max. Value attK_c :195.50318908691398
Min. Value attK_c :-48.555568695068395
Entt Desc. Potassium count, attenuated. Limit high-frequency over-attenuation >265m STP on [attK_o].
Att Unit cps

attK_o: 1
Max. Value attK_o :229.601470947266
Min. Value attK_o :-122.68441772460899
Entt Desc. Potassium count, attenuated. Attenuation to 100m, final calibrations used.
Att Unit cps

attTC_c: 2
Max. Value attTC_c :1648.55895996094
Min. Value attTC_c :61.4421577453613
Entt Desc. Total count, attenuated. Limit high-frequency over-attenuation >265m STP on [attTC_o].
Att Unit cps

attTC_o: 3
Max. Value attTC_o :1671.8977050781302
Min. Value attTC_o :61.4421577453613
Entt Desc. Air-absorbed dose rate. Computed using sensitivity at 100m from [attTC_o].
Att Unit nGy/h

attTH_c: 4
Max. Value attTH_c :79.9993057250977
Min. Value attTH_c :-13.3069620132446
Entt Desc. Thorium count, attenuated.  Limit high-frequency over-attenuation >265m STP on [attTH_o].
Att Unit cps

attTH_o: 5
Max. Value attT

Max. Value Templc :29.073490142822298
Min. Value Templc :-8.792722702026369
Entt Desc. Temperature, edited. Measured temperature, edited against spikes and jumps.
Att Unit degrees centigrade

THKratio: 55
Max. Value THKratio :43.096858978271506
Min. Value THKratio :1.44411730766296
Entt Desc. Uranium/thorium ratio [conU_f/conTH_f], canceled for aberrant values.
Att Unit ppm/ppm

THraw: 56
Max. Value THraw :71
Min. Value THraw :1
Entt Desc. Thorium count, raw. 33.6L DOWN, ch. 0803-0937/1024, window 2410-2810 keV
Att Unit cps

topoSRTM: 57
Max. Value topoSRTM :788.2978515625
Min. Value topoSRTM :62.067501068115206
Entt Desc. Published terrain model MSL, DTM model, Shuttle Radar Topography Model.
Att Unit meters

UKratio: 58
Max. Value UKratio :12.4528121948242
Min. Value UKratio :-0.0238849967718124
Entt Desc. Uranium/potassium ratio [conU_f/conK_f], canceled for aberrant values.
Att Unit ppm/percent

upUraw: 59
Max. Value upUraw :24
Min. Value upUraw :0
Entt Desc. UP-uranium count, raw.

### Now make the EandA for the seperate spectra files

# *D-RAW

In [146]:
#Set Data Paths - perhaps we'll get a user form to do this some day?
# The r signifies a string literal. Use for paths.

drawCSVfileDir = r"C:\CurrentWork\DataReleases\NorthwesternArkansas\CSV_Data"
drawCSVfileName = "USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricSpecDrawWaveforms.csv"
#========================================================================================================================

drawCSVfilePath = drawCSVfileDir + '\\' + drawCSVfileName
print ('The Draw CSV Data Path is: ' + '"' + drawCSVfilePath + '"')#magCSVfilePath

The Draw CSV Data Path is: "C:\CurrentWork\DataReleases\NorthwesternArkansas\CSV_Data\USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricSpecDrawWaveforms.csv"


In [56]:
# read file into pandas; note that can take over 1 minute depending on the size of the CSV file and computing power.
print ('!!! PLEASE BE PATIENT AND TAKE SEVERAL BREATHS !!!')
print ('Reading file into Pandas, this may take several minutes depending on the file size...')

drawDataFrame = pd.read_csv(drawCSVfilePath, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
drawDataFrame

!!! PLEASE BE PATIENT AND TAKE SEVERAL BREATHS !!!
Reading file into Pandas, this may take several minutes depending on the file size...


,Line,fiducial,specDraw[0],specDraw[1],specDraw[2],specDraw[3],specDraw[4],specDraw[5],specDraw[6],specDraw[7],...,specDraw[1014],specDraw[1015],specDraw[1016],specDraw[1017],specDraw[1018],specDraw[1019],specDraw[1020],specDraw[1021],specDraw[1022],specDraw[1023]
0,L20010,48639,0,0,0,1,1,11,18,10,...,0,0,0,0,0,0,0,0,0,153
1,L20010,48640,0,0,0,1,4,12,16,6,...,0,0,0,0,0,0,0,0,0,168
2,L20010,48641,0,0,0,2,1,15,15,15,...,0,0,0,0,0,0,0,1,0,174
3,L20010,48642,0,0,0,0,10,11,13,12,...,0,0,0,0,0,0,0,1,0,183
4,L20010,48643,0,0,0,1,10,9,11,15,...,0,0,0,0,0,0,0,0,0,160
5,L20010,48644,0,0,0,2,9,7,11,11,...,0,0,0,1,0,1,0,0,0,155
6,L20010,48645,0,0,0,3,9,11,12,14,...,0,0,1,0,0,0,0,0,0,151
7,L20010,48646,0,0,0,0,6,12,18,13,...,0,0,0,0,0,0,0,0,0,168
8,L20010,48647,0,0,0,1,6,16,17,11,...,0,0,0,0,0,0,0,0,0,146
9,L20010,48648,0,0,0,4,5,14,11,14,...,0,0,0,0,0,0,0,0,0,167


In [147]:
# now get an array of the header names
drawHeaderList = list(drawDataFrame.columns.values)
drawHeaderString = ''
for i in range(len(drawHeaderList)):
 drawHeaderString = drawHeaderString + drawHeaderList[i] + ','

print (drawHeaderString)

Line,fiducial,specDraw[0],specDraw[1],specDraw[2],specDraw[3],specDraw[4],specDraw[5],specDraw[6],specDraw[7],specDraw[8],specDraw[9],specDraw[10],specDraw[11],specDraw[12],specDraw[13],specDraw[14],specDraw[15],specDraw[16],specDraw[17],specDraw[18],specDraw[19],specDraw[20],specDraw[21],specDraw[22],specDraw[23],specDraw[24],specDraw[25],specDraw[26],specDraw[27],specDraw[28],specDraw[29],specDraw[30],specDraw[31],specDraw[32],specDraw[33],specDraw[34],specDraw[35],specDraw[36],specDraw[37],specDraw[38],specDraw[39],specDraw[40],specDraw[41],specDraw[42],specDraw[43],specDraw[44],specDraw[45],specDraw[46],specDraw[47],specDraw[48],specDraw[49],specDraw[50],specDraw[51],specDraw[52],specDraw[53],specDraw[54],specDraw[55],specDraw[56],specDraw[57],specDraw[58],specDraw[59],specDraw[60],specDraw[61],specDraw[62],specDraw[63],specDraw[64],specDraw[65],specDraw[66],specDraw[67],specDraw[68],specDraw[69],specDraw[70],specDraw[71],specDraw[72],specDraw[73],specDraw[74],specDraw[75],specDraw

In [148]:
# Now make EandA for the Down-Raw Spectra
drawAttributes = ''
drawEntityDescription = []
drawEntityUnit = []
csvMax = []
csvMin = []
#=======================================================================================================================
# Below is the entity type description for the Down-Raw spectrum waveforms
drawEntAttText = '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>' + drawCSVfileName + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>raw radiometric spectrum down waveforms.' \
            +'  These are presented in a data array channel of 1024 samples indexed to the '\
            +  radCSVfileName + '</enttypd>' \
            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>'
#=======================================================================================================================
drawEntityDescription.append("Line")#line
drawEntityUnit.append("llln")#line
#=======================================================================================================================
drawEntityDescription.append("Fiducial; Montaj/RMS system fiducial.")#fid01
drawEntityUnit.append("sec UTC")#fid01
#=======================================================================================================================


for j in range(0, 1024):    
 drawEntityDescription.append("Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.")#specDraw
 drawEntityUnit.append("cps")#specDraw
 
drawHeaderListLength = len(drawEntityDescription)
print ('Length of Entity Description Array: ' + str(len(drawEntityDescription)))
print ('Length of Header List: ' + str(len(drawHeaderList)))
for i in range(drawHeaderListLength): 
 print (drawHeaderList[i] + ': ' + str(i))
 strCol = drawHeaderList[i]
 csvMax.append(drawDataFrame[strCol].max())
 csvMin.append(drawDataFrame[strCol].min())
 print ('Max. Value ' + drawHeaderList[i] + ' :' + str(csvMax[i]))
 print ('Min. Value ' + drawHeaderList[i] + ' :' + str(csvMin[i]))
 print ('Entt Desc. ' + drawEntityDescription[i])
 print ('Att Unit ' + drawEntityUnit[i] + '\n')
 
 entAtti = '\n\t\t\t\t<attr>\n\t\t\t\t\t<attrlabl>' + strCol + '</attrlabl>\n' \
        + '\t\t\t\t\t<attrdef>' + drawEntityDescription[i] + '</attrdef>' \
        + '\n\t\t\t\t\t<attrdefs>EON GEOSCIENCES INC.</attrdefs>' \
        + '\n\t\t\t\t\t<attrdomv>\n\t\t\t\t\t\t<rdom>\n\t\t\t\t\t\t\t<rdommin>'+ str(csvMin[i]) + '</rdommin>\n\t\t\t\t\t\t\t<rdommax>' \
        + str(csvMax[i]) + '</rdommax>\n\t\t\t\t\t\t\t' + '<attrunit>' + drawEntityUnit[i] + '</attrunit>\n\t\t\t\t\t\t</rdom>\n\t\t\t\t\t' \
        + '</attrdomv>\n' \
        + '\t\t\t\t</attr>'
 drawEntAttText = drawEntAttText + entAtti

drawEntAttText = drawEntAttText + '\n\t\t</detailed>\n'  
            

print ('\nEntity and Attribute Text:\n' + drawEntAttText)

Length of Entity Description Array: 1026
Length of Header List: 1026
Line: 0
Max. Value Line :T50370
Min. Value Line :L20010
Entt Desc. Line
Att Unit llln

fiducial: 1
Max. Value fiducial :87331
Min. Value fiducial :48639
Entt Desc. Fiducial; Montaj/RMS system fiducial.
Att Unit sec UTC

specDraw[0]: 2
Max. Value specDraw[0] :0
Min. Value specDraw[0] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[1]: 3
Max. Value specDraw[1] :0
Min. Value specDraw[1] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[2]: 4
Max. Value specDraw[2] :0
Min. Value specDraw[2] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[3]: 5
Max. Value specDraw[3] :19
Min. Value specDraw[3] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.


Max. Value specDraw[48] :84
Min. Value specDraw[48] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[49]: 51
Max. Value specDraw[49] :80
Min. Value specDraw[49] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[50]: 52
Max. Value specDraw[50] :79
Min. Value specDraw[50] :1
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[51]: 53
Max. Value specDraw[51] :75
Min. Value specDraw[51] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[52]: 54
Max. Value specDraw[52] :83
Min. Value specDraw[52] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[53]: 55
Max. Value specDraw[53] :76
Min. Value specDraw[53] :0
Entt Desc. Spec

Max. Value specDraw[94] :38
Min. Value specDraw[94] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[95]: 97
Max. Value specDraw[95] :42
Min. Value specDraw[95] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[96]: 98
Max. Value specDraw[96] :39
Min. Value specDraw[96] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[97]: 99
Max. Value specDraw[97] :35
Min. Value specDraw[97] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[98]: 100
Max. Value specDraw[98] :37
Min. Value specDraw[98] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[99]: 101
Max. Value specDraw[99] :36
Min. Value specDraw[99] :0
Entt Desc. Sp

Max. Value specDraw[140] :20
Min. Value specDraw[140] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[141]: 143
Max. Value specDraw[141] :22
Min. Value specDraw[141] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[142]: 144
Max. Value specDraw[142] :24
Min. Value specDraw[142] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[143]: 145
Max. Value specDraw[143] :23
Min. Value specDraw[143] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[144]: 146
Max. Value specDraw[144] :23
Min. Value specDraw[144] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[145]: 147
Max. Value specDraw[145] :21
Min. Value specDraw[1

Max. Value specDraw[187] :18
Min. Value specDraw[187] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[188]: 190
Max. Value specDraw[188] :18
Min. Value specDraw[188] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[189]: 191
Max. Value specDraw[189] :18
Min. Value specDraw[189] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[190]: 192
Max. Value specDraw[190] :19
Min. Value specDraw[190] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[191]: 193
Max. Value specDraw[191] :18
Min. Value specDraw[191] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[192]: 194
Max. Value specDraw[192] :19
Min. Value specDraw[1

...

Max. Value specDraw[788] :5
Min. Value specDraw[788] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[789]: 791
Max. Value specDraw[789] :4
Min. Value specDraw[789] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[790]: 792
Max. Value specDraw[790] :4
Min. Value specDraw[790] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[791]: 793
Max. Value specDraw[791] :4
Min. Value specDraw[791] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[792]: 794
Max. Value specDraw[792] :5
Min. Value specDraw[792] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[793]: 795
Max. Value specDraw[793] :5
Min. Value specDraw[793] :0

Max. Value specDraw[836] :4
Min. Value specDraw[836] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[837]: 839
Max. Value specDraw[837] :4
Min. Value specDraw[837] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[838]: 840
Max. Value specDraw[838] :4
Min. Value specDraw[838] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[839]: 841
Max. Value specDraw[839] :4
Min. Value specDraw[839] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[840]: 842
Max. Value specDraw[840] :4
Min. Value specDraw[840] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[841]: 843
Max. Value specDraw[841] :4
Min. Value specDraw[841] :0

Max. Value specDraw[886] :6
Min. Value specDraw[886] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[887]: 889
Max. Value specDraw[887] :5
Min. Value specDraw[887] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[888]: 890
Max. Value specDraw[888] :7
Min. Value specDraw[888] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[889]: 891
Max. Value specDraw[889] :6
Min. Value specDraw[889] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[890]: 892
Max. Value specDraw[890] :6
Min. Value specDraw[890] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[891]: 893
Max. Value specDraw[891] :6
Min. Value specDraw[891] :0

Max. Value specDraw[937] :3
Min. Value specDraw[937] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[938]: 940
Max. Value specDraw[938] :3
Min. Value specDraw[938] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[939]: 941
Max. Value specDraw[939] :3
Min. Value specDraw[939] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[940]: 942
Max. Value specDraw[940] :3
Min. Value specDraw[940] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[941]: 943
Max. Value specDraw[941] :3
Min. Value specDraw[941] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[942]: 944
Max. Value specDraw[942] :3
Min. Value specDraw[942] :0

Max. Value specDraw[991] :3
Min. Value specDraw[991] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[992]: 994
Max. Value specDraw[992] :3
Min. Value specDraw[992] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[993]: 995
Max. Value specDraw[993] :3
Min. Value specDraw[993] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[994]: 996
Max. Value specDraw[994] :3
Min. Value specDraw[994] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[995]: 997
Max. Value specDraw[995] :3
Min. Value specDraw[995] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specDraw[996]: 998
Max. Value specDraw[996] :3
Min. Value specDraw[996] :0

# *U-RAW

In [93]:
#Set Data Paths - perhaps we'll get a user form to do this some day?
# The r signifies a string literal. Use for paths.

urawCSVfileDir = r"C:\CurrentWork\DataReleases\NorthwesternArkansas\CSV_Data"
urawCSVfileName = "USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricSpecUrawWaveforms.csv"
#========================================================================================================================

urawCSVfilePath = urawCSVfileDir + '\\' + urawCSVfileName
print ('The U-raw CSV Data Path is: ' + '"' + urawCSVfilePath + '"')#UrawCSVfilePath

The U-raw CSV Data Path is: "C:\CurrentWork\DataReleases\NorthwesternArkansas\CSV_Data\USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricSpecUrawWaveforms.csv"


In [60]:
# read file into pandas; note that can take over 1 minute depending on the size of the CSV file and computing power.
print ('!!! PLEASE BE PATIENT AND TAKE SEVERAL BREATHS !!!')
print ('Reading file into Pandas, this may take several minutes depending on the file size...')

urawDataFrame = pd.read_csv(urawCSVfilePath, sep=',', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
urawDataFrame

!!! PLEASE BE PATIENT AND TAKE SEVERAL BREATHS !!!
Reading file into Pandas, this may take several minutes depending on the file size...


,Line,fiducial,specUraw[0],specUraw[1],specUraw[2],specUraw[3],specUraw[4],specUraw[5],specUraw[6],specUraw[7],...,specUraw[1014],specUraw[1015],specUraw[1016],specUraw[1017],specUraw[1018],specUraw[1019],specUraw[1020],specUraw[1021],specUraw[1022],specUraw[1023]
0,L20010,48639,0,0,0,0,2,3,4,1,...,0,0,0,0,0,0,0,0,0,47
1,L20010,48640,0,0,0,0,1,5,4,5,...,0,0,0,0,0,0,0,0,0,38
2,L20010,48641,0,0,0,0,0,2,2,4,...,0,0,0,0,0,0,0,0,0,46
3,L20010,48642,0,0,0,0,4,5,4,3,...,0,0,0,0,0,0,0,0,0,36
4,L20010,48643,0,0,0,0,0,3,2,3,...,0,0,0,0,0,0,0,0,0,41
5,L20010,48644,0,0,0,0,1,5,0,3,...,0,0,0,0,0,0,0,0,0,41
6,L20010,48645,0,0,0,0,4,3,6,6,...,0,0,0,0,0,0,0,1,0,50
7,L20010,48646,0,0,0,0,3,2,3,7,...,0,0,0,0,0,0,0,0,0,46
8,L20010,48647,0,0,0,0,0,4,1,5,...,0,0,0,0,0,0,0,0,0,55
9,L20010,48648,0,0,0,0,1,2,4,3,...,0,0,0,0,0,0,0,0,0,32


In [149]:
# now get an array of the header names
urawHeaderList = list(urawDataFrame.columns.values)
urawHeaderString = ''
for i in range(len(urawHeaderList)):
 urawHeaderString = urawHeaderString + urawHeaderList[i] + ','

print (urawHeaderString)

Line,fiducial,specUraw[0],specUraw[1],specUraw[2],specUraw[3],specUraw[4],specUraw[5],specUraw[6],specUraw[7],specUraw[8],specUraw[9],specUraw[10],specUraw[11],specUraw[12],specUraw[13],specUraw[14],specUraw[15],specUraw[16],specUraw[17],specUraw[18],specUraw[19],specUraw[20],specUraw[21],specUraw[22],specUraw[23],specUraw[24],specUraw[25],specUraw[26],specUraw[27],specUraw[28],specUraw[29],specUraw[30],specUraw[31],specUraw[32],specUraw[33],specUraw[34],specUraw[35],specUraw[36],specUraw[37],specUraw[38],specUraw[39],specUraw[40],specUraw[41],specUraw[42],specUraw[43],specUraw[44],specUraw[45],specUraw[46],specUraw[47],specUraw[48],specUraw[49],specUraw[50],specUraw[51],specUraw[52],specUraw[53],specUraw[54],specUraw[55],specUraw[56],specUraw[57],specUraw[58],specUraw[59],specUraw[60],specUraw[61],specUraw[62],specUraw[63],specUraw[64],specUraw[65],specUraw[66],specUraw[67],specUraw[68],specUraw[69],specUraw[70],specUraw[71],specUraw[72],specUraw[73],specUraw[74],specUraw[75],specUraw

In [163]:
# Now make EandA for the D-Raw Spectra
urawAttributes = ''
urawEntityDescription = []
urawEntityUnit = []
csvMax = []
csvMin = []

#=======================================================================================================================
# Below is the entity type description for the Up-Raw spectrum waveforms
urawEntAttText = '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>' + urawCSVfileName + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>raw radiometric spectrum up waveforms.' \
            +'  These are presented in a data array channel of 1024 samples indexed to the '\
            +  radCSVfileName + '</enttypd>' \
            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>'

#=======================================================================================================================
urawEntityDescription.append("Line")#line
urawEntityUnit.append("llln")#line
#=======================================================================================================================
urawEntityDescription.append("Fiducial; Montaj/RMS system fiducial.")#fid01
urawEntityUnit.append("sec UTC")#fid01
#=======================================================================================================================

for j in range(0, 1024):    
 urawEntityDescription.append("Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.")#specDraw
 urawEntityUnit.append("cps")#specUraw
 
urawHeaderListLength = len(drawEntityDescription)
print ('Length of Entity Description Array: ' + str(len(urawEntityDescription)))
print ('Length of Header List: ' + str(len(urawHeaderList)))
for i in range(urawHeaderListLength):
 print (urawHeaderList[i] + ': ' + str(i))
 strCol = urawHeaderList[i]
 csvMax.append(urawDataFrame[strCol].max())
 csvMin.append(urawDataFrame[strCol].min())
 print ('Max. Value ' + urawHeaderList[i] + ' :' + str(csvMax[i]))
 print ('Min. Value ' + urawHeaderList[i] + ' :' + str(csvMin[i]))
 print ('Entt Desc. ' + urawEntityDescription[i])
 print ('Att Unit ' + urawEntityUnit[i] + '\n')
 
 entAtti = '\n\t\t\t\t<attr>\n\t\t\t\t\t<attrlabl>' + strCol + '</attrlabl>\n' \
        + '\t\t\t\t\t<attrdef>' + urawEntityDescription[i] + '</attrdef>' \
        + '\n\t\t\t\t\t<attrdefs>EON GEOSCIENCES INC.</attrdefs>' \
        + '\n\t\t\t\t\t<attrdomv>\n\t\t\t\t\t\t<rdom>\n\t\t\t\t\t\t\t<rdommin>'+ str(csvMin[i]) + '</rdommin>\n\t\t\t\t\t\t\t<rdommax>' \
        + str(csvMax[i]) + '</rdommax>\n\t\t\t\t\t\t\t' + '<attrunit>' + urawEntityUnit[i] + '</attrunit>\n\t\t\t\t\t\t</rdom>\n\t\t\t\t\t' \
        + '</attrdomv>\n' \
        + '\t\t\t\t</attr>'
 urawEntAttText = urawEntAttText + entAtti

urawEntAttText = urawEntAttText + '\n\t\t</detailed>\n' 

print ('\nEntity and Attribute Text:\n' + urawEntAttText)

Length of Entity Description Array: 1026
Length of Header List: 1026
Line: 0
Max. Value Line :T50370
Min. Value Line :L20010
Entt Desc. Line
Att Unit llln

fiducial: 1
Max. Value fiducial :87331
Min. Value fiducial :48639
Entt Desc. Fiducial; Montaj/RMS system fiducial.
Att Unit sec UTC

specUraw[0]: 2
Max. Value specUraw[0] :0
Min. Value specUraw[0] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[1]: 3
Max. Value specUraw[1] :0
Min. Value specUraw[1] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[2]: 4
Max. Value specUraw[2] :0
Min. Value specUraw[2] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[3]: 5
Max. Value specUraw[3] :5
Min. Value specUraw[3] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
A

Max. Value specUraw[50] :26
Min. Value specUraw[50] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[51]: 53
Max. Value specUraw[51] :26
Min. Value specUraw[51] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[52]: 54
Max. Value specUraw[52] :24
Min. Value specUraw[52] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[53]: 55
Max. Value specUraw[53] :23
Min. Value specUraw[53] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[54]: 56
Max. Value specUraw[54] :22
Min. Value specUraw[54] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[55]: 57
Max. Value specUraw[55] :22
Min. Value specUraw[55] :0
Entt Desc. Spec

Max. Value specUraw[103] :11
Min. Value specUraw[103] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[104]: 106
Max. Value specUraw[104] :13
Min. Value specUraw[104] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[105]: 107
Max. Value specUraw[105] :12
Min. Value specUraw[105] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[106]: 108
Max. Value specUraw[106] :11
Min. Value specUraw[106] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[107]: 109
Max. Value specUraw[107] :11
Min. Value specUraw[107] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[108]: 110
Max. Value specUraw[108] :12
Min. Value specUraw[1

Max. Value specUraw[151] :8
Min. Value specUraw[151] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[152]: 154
Max. Value specUraw[152] :8
Min. Value specUraw[152] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[153]: 155
Max. Value specUraw[153] :7
Min. Value specUraw[153] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[154]: 156
Max. Value specUraw[154] :9
Min. Value specUraw[154] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[155]: 157
Max. Value specUraw[155] :8
Min. Value specUraw[155] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[156]: 158
Max. Value specUraw[156] :8
Min. Value specUraw[156] :0

Max. Value specUraw[197] :8
Min. Value specUraw[197] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[198]: 200
Max. Value specUraw[198] :8
Min. Value specUraw[198] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[199]: 201
Max. Value specUraw[199] :8
Min. Value specUraw[199] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[200]: 202
Max. Value specUraw[200] :9
Min. Value specUraw[200] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[201]: 203
Max. Value specUraw[201] :8
Min. Value specUraw[201] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[202]: 204
Max. Value specUraw[202] :8
Min. Value specUraw[202] :0

...

Max. Value specUraw[787] :2
Min. Value specUraw[787] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[788]: 790
Max. Value specUraw[788] :3
Min. Value specUraw[788] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[789]: 791
Max. Value specUraw[789] :3
Min. Value specUraw[789] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[790]: 792
Max. Value specUraw[790] :3
Min. Value specUraw[790] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[791]: 793
Max. Value specUraw[791] :3
Min. Value specUraw[791] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[792]: 794
Max. Value specUraw[792] :2
Min. Value specUraw[792] :0

Max. Value specUraw[840] :3
Min. Value specUraw[840] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[841]: 843
Max. Value specUraw[841] :3
Min. Value specUraw[841] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[842]: 844
Max. Value specUraw[842] :2
Min. Value specUraw[842] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[843]: 845
Max. Value specUraw[843] :3
Min. Value specUraw[843] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[844]: 846
Max. Value specUraw[844] :2
Min. Value specUraw[844] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[845]: 847
Max. Value specUraw[845] :2
Min. Value specUraw[845] :0

Max. Value specUraw[893] :2
Min. Value specUraw[893] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[894]: 896
Max. Value specUraw[894] :3
Min. Value specUraw[894] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[895]: 897
Max. Value specUraw[895] :3
Min. Value specUraw[895] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[896]: 898
Max. Value specUraw[896] :3
Min. Value specUraw[896] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[897]: 899
Max. Value specUraw[897] :2
Min. Value specUraw[897] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[898]: 900
Max. Value specUraw[898] :2
Min. Value specUraw[898] :0

Max. Value specUraw[947] :3
Min. Value specUraw[947] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[948]: 950
Max. Value specUraw[948] :2
Min. Value specUraw[948] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[949]: 951
Max. Value specUraw[949] :2
Min. Value specUraw[949] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[950]: 952
Max. Value specUraw[950] :3
Min. Value specUraw[950] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[951]: 953
Max. Value specUraw[951] :2
Min. Value specUraw[951] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[952]: 954
Max. Value specUraw[952] :2
Min. Value specUraw[952] :0

Max. Value specUraw[1001] :2
Min. Value specUraw[1001] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[1002]: 1004
Max. Value specUraw[1002] :2
Min. Value specUraw[1002] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[1003]: 1005
Max. Value specUraw[1003] :2
Min. Value specUraw[1003] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[1004]: 1006
Max. Value specUraw[1004] :2
Min. Value specUraw[1004] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[1005]: 1007
Max. Value specUraw[1005] :2
Min. Value specUraw[1005] :0
Entt Desc. Spectrum down amplitude of sample number indicated. Array channel for 1 of 1024 samples.
Att Unit cps

specUraw[1006]: 1008
Max. Value specUraw[1006] :3
Min. V

# Now lets create Entity and Attributes for the rest of the files that don't require listing the min/max for the domains.

In [164]:
#Get a listing of the tif files
#strip Tabs from string
stripFileListing = fileListing.replace('\t', '')
#Create array of values by splitting on line return \n
arrayFileListing = stripFileListing.splitlines()
#arrayFileListing[0] = 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_TMFI.tif'
arrayFileListing

['USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqPotassium.tif',
 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqTC.tif',
 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqThorium.tif',
 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqUranium.tif',
 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_UThK-TertiaryImage.tif']

In [165]:
# Create the Entity and Attributes for the other files being released besides the *.CSV data file
bFilesEntAttText = ''
bFilesEandA = ''
for i in range(len(arrayFileListing)):
    if arrayFileListing[i].find('.tif') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>3-band raster geospatial data file of the aeromagnetic data.</enttypd>' \
            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Value</attrlabl>' \
            + '\n\t\t\t\t<attrdef>Unique numeric values contained in each raster cell.</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<rdom>\n\t\t\t\t\t\t<rdommin>0</rdommin>\n\t\t\t\t\t\t<rdommax>255</rdommax>'\
            + '\n\t\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n' 
    if arrayFileListing[i].find('.rtf') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Rich Text File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ReadMe file in Rich Text Format containing *.CSV file entities, attributes, and processing notes</enttypd>' \
            + '\n\t\t\t\t<enttypds>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Rich Text File</attrlabl>\n\t\t\t\t<attrdef>ReadMe file in Rich Text Format (RTF) containing *.CSV file entities, attributes, and processing notes</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>ReadMe file in Rich Text Format containing *.CSV file entities, attributes, and processing notes</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    if arrayFileListing[i].find('.gdb') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Binary File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Binary data base file in Geosoft (https://www.geosoft.com) Oasis montaj GRD format.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Binary data base file</attrlabl>\n\t\t\t\t<attrdef>Binary data base file described in contractor report and ReadMe file</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>Binary Data Base File described in contractor report and ReadMe file</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if arrayFileListing[i].find('.xml') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>ASCII XML File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ASCII text XML formatted metadata file describing the data base file of the same name.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>ASCII XML formatted text file</attrlabl>\n\t\t\t\t<attrdef>ASCII XML formatted text file containing metadata describing the Geosoft data base file of the same name</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n\t\t\t\t<attrdomv>'\
            + '\n\t\t\t\t\t<udom>ASCII XML formatted text file containing metadata describing the Geosoft data base file of the same name</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if arrayFileListing[i].find('.gxf') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>ASCII Grid File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>ASCII grid file in Geosoft (https://www.geosoft.com) Grid Exchange Format, GXF (https://www.geosoft.com/media/uploads/resources/technical-notes/gxfr3d9_1.pdf) </enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>ASCII Grid File</attrlabl>\n\t\t\t\t<attrdef>ASCII grid file in GXF format described in contractor report and ReadMe file, WGS84 UTM Zone 13N</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n'\
            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>ASCII Grid File described in contractor report and ReadMe file, WGS84 UTM Zone 13N</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    
    if arrayFileListing[i].find('.grd') != -1:
        bFilesEandA = bFilesEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Binary Grid File ' + arrayFileListing[i] + '</enttypl>\n' \
            + '\t\t\t\t<enttypd>Binary grid file in Geosoft (https://www.geosoft.com) Oasis montaj GRD format.  A free viewer is available for GRD files from Geosoft, visit https://www.geosoft.com/products/geosoft-viewer</enttypd>' \
            + '\n\t\t\t\t<enttypds>Geosoft Inc., Toronto, ON Canada</enttypds>\n\t\t\t</enttyp>' \
            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Binary grid file</attrlabl>\n\t\t\t\t<attrdef>Binary grid file described in contractor report and ReadMe file, WGS84 UTM Zone 13N</attrdef>\n\t\t\t\t<attrdefs>EON Geosciences, Montreal, Quebec, Canada: Airborne geophysical contractor for the U.S. Geological Survey.</attrdefs>\n'\
            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>Binary Grid File described in contractor report and ReadMe file, WGS84 UTM Zone 13N</udom>\n\t\t\t\t</attrdomv>' \
            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    bFilesEntAttText = bFilesEandA

print (bFilesEntAttText) 


		<detailed>
			<enttyp>
				<enttypl>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqPotassium.tif</enttypl>
				<enttypd>3-band raster geospatial data file of the aeromagnetic data.</enttypd>
				<enttypds>U.S. Geological Survey</enttypds>
			</enttyp>
			<attr>
				<attrlabl>Value</attrlabl>
				<attrdef>Unique numeric values contained in each raster cell.</attrdef>
				<attrdefs>U.S. Geological Survey</attrdefs>
				<attrdomv>
					<rdom>
						<rdommin>0</rdommin>
						<rdommax>255</rdommax>
					</rdom>
				</attrdomv>
			</attr>
		</detailed>
		<detailed>
			<enttyp>
				<enttypl>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_GeoreferencedRaster_EqTC.tif</enttypl>
				<enttypd>3-band raster geospatial data file of the aeromagnetic data.</enttypd>
				<enttypds>U.S. Geological Survey</enttypds>
			</enttyp>
			<attr>
				<attrlabl>Value</attrlabl>
				<attrdef>Unique numeric values contained in each raster cell.</attrdef>
				<attrdefs>U.S. Geological Su

In [166]:
#Enter the URL for the radiometric data child for this data release between single quotes below

###############################################################
childURL = 'https://www.sciencebase.gov/catalog/item/5f068fc082ce21d4c3f90a35'
###############################################################
print (childURL)

https://www.sciencebase.gov/catalog/item/5f068fc082ce21d4c3f90a35


In [167]:
#Now make the EandA for the png file:
#Enter the png file name and path
#########################################################################################################################
pngFileName = 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Raster_EqTotalCounts.png'
pngDirPath = r'C:\CurrentWork\DataReleases\NorthwesternArkansas\PNGs'
#########################################################################################################################

pngFilePath = pngDirPath + "\\" + pngFileName
txtListFormatted = []
txtListFormatted.append(pngFileName)#png file

print('pngFilePath\n' + pngFilePath)


pngFilePath
C:\CurrentWork\DataReleases\NorthwesternArkansas\PNGs\USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Raster_EqTotalCounts.png


In [168]:
# The code below can be cleaned up to handle multiple files with differnt extensions

txtEandA = ''
#As before, we read in the list of files

for i in range(len(txtListFormatted)):
    if txtListFormatted[i].find('png') != -1:

        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>' + txtListFormatted[i] + ' is a Portable Network Graphics (PNG) image associated with the data set.</eaover> '\
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \

    if txtListFormatted[i].find('pdf') != -1:

        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File. ' + txtListFormatted[i] + ' is a header file in ASCII text format for binary Fourier coefficient files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \

    if txtListFormatted[i].find('-SD') != -1:
        
        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File, ' + txtListFormatted[i] + ' is a header file in ASCII text format for synchronous detection cross power files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \

            
    if txtListFormatted[i].find('-TS') != -1:

        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File, ' + txtListFormatted[i] + ' is a header file in ASCII text format for binary time series files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \

#Add the final .txt listing for the readme file.  
#Be sure to comment this out if there is no readme.txt file included with the data release

##readmeEandA = '\t\t<overview>\n\t\t<eaover>Text File, readme.txt, is a Read Me file describing the naming format of the EDI files and that they may need to be renamed to be imported into certain software packages.</eaover>'\
           ## + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
    
txtEandA = txtEandA #+ readmeEandA
print (txtEandA)
#ALL_EandA = '' #'<eainfo>\n'   
#ALL_EandA = ALL_EandA + txtEandA

		<overview>
			<eaover>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Raster_EqTotalCounts.png is a Portable Network Graphics (PNG) image associated with the data set.</eaover> 
			<eadetcit>https://www.sciencebase.gov/catalog/item/5f068fc082ce21d4c3f90a35</eadetcit>
		</overview>



# Now concatenate all the entity and attribute text


In [169]:
eainfo = csvAttributes + drawEntAttText + drawEntAttText + bFilesEntAttText + txtEandA
print (eainfo)

		<detailed>
			<enttyp>
				<enttypl>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricData-RadSpecCulled.csv</enttypl>
				<enttypd>Table containing attribute information associated with the data set.</enttypd>
				<enttypds>U.S. Geological Survey</enttypds>
			</enttyp>
				<attr>
					<attrlabl>attK_c</attrlabl>
					<attrdef>Potassium count, attenuated. Limit high-frequency over-attenuation >265m STP on [attK_o].</attrdef>
					<attrdefs>EON GEOSCIENCES INC.</attrdefs>
					<attrdomv>
						<rdom>
							<rdommin>-48.555568695068395</rdommin>
							<rdommax>195.50318908691398</rdommax>
							<attrunit>cps</attrunit>
						</rdom>
						</attrdomv>
				</attr>
				<attr>
					<attrlabl>attK_o</attrlabl>
					<attrdef>Potassium count, attenuated. Attenuation to 100m, final calibrations used.</attrdef>
					<attrdefs>EON GEOSCIENCES INC.</attrdefs>
					<attrdomv>
						<rdom>
							<rdommin>-122.68441772460899</rdommin>
							<rdommax>229.601470947266</rdommax>
							

## Create the stdorder for all the non-extension (tiff) files 

In [170]:
#Enter form content description for files
#########################################################################################################################
csvFormCont = 'Comma Seperated Values (CSV) ASCII tabular data file containing attribute information associated with the data set.'
csvFormName = 'CSV' # See https://www.fgdc.gov/csdgmgraphical/distr/storder/digform/traninf/fname.htm for acceptable text values
pngFormCont = 'Portable Network Graphics (PNG) raster image associated with the data set.'
pngFormName = 'PNG' # See https://www.fgdc.gov/csdgmgraphical/distr/storder/digform/traninf/fname.htm for acceptable text values
#########################################################################################################################

# Get file sizes
csvFilesize = os.path.getsize(radCSVfilePath)
csvFilesize = csvFilesize/1000000 # convert file size from bytes to megabytes
pngFilesize = os.path.getsize(pngFilePath)
pngFilesize = pngFilesize/1000000 # convert file size from bytes to megabytes
arrStdOrder = [[radCSVfileName, csvFormCont, csvFilesize, csvFormName],[pngFileName, pngFormCont, pngFilesize, pngFormName]]
            
txtStdOrder = '\t\t<stdorder>'
for i in range(len(arrStdOrder)):
 txtStdOrder = txtStdOrder + '\n\t\t\t<digform>' +  '\n\t\t\t\t<digtinfo>' \
    + '\n\t\t\t\t\t<formname>' + str(arrStdOrder[i][3]) + '</formname>' \
    + '\n\t\t\t\t\t<formvern>' + '1' + '</formvern>' \
    + '\n\t\t\t\t\t<formspec>' + str(arrStdOrder[i][0]) + '</formspec>' \
    + '\n\t\t\t\t\t<formcont>' + str(arrStdOrder[i][1]) + '</formcont>' \
    + '\n\t\t\t\t\t<filedec>' + 'No compression applied </filedec>' \
    + '\n\t\t\t\t\t<transize>' + str(arrStdOrder[i][2]) + '</transize>\n\t\t\t\t' \
    + '</digtinfo>' + '\n\t\t\t<digtopt>' + '\n\t\t\t\t<onlinopt>' + '\n\t\t\t\t\t<computer>' + '\n\t\t\t\t\t\t<networka>' \
    + '\n\t\t\t\t\t\t\t<networkr>' + childURL + '</networkr>' + '\n\t\t\t\t\t\t</networka>' \
    + '\n\t\t\t\t\t\t</computer>' + '\n\t\t\t\t</onlinopt>' + '\n\t\t\t</digtopt>\n\t\t</digform>'
    
txtStdOrder = txtStdOrder  + '\n\t\t\t\t\t\t<fees>' + 'none' + '</fees>' + '\n\t\t</stdorder>'

stdorder = txtStdOrder
print(str(stdorder))

		<stdorder>
			<digform>
				<digtinfo>
					<formname>CSV</formname>
					<formvern>1</formvern>
					<formspec>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_RadiometricData-RadSpecCulled.csv</formspec>
					<formcont>Comma Seperated Values (CSV) ASCII tabular data file containing attribute information associated with the data set.</formcont>
					<filedec>No compression applied </filedec>
					<transize>757.973278</transize>
				</digtinfo>
			<digtopt>
				<onlinopt>
					<computer>
						<networka>
							<networkr>https://www.sciencebase.gov/catalog/item/5f068fc082ce21d4c3f90a35</networkr>
						</networka>
						</computer>
				</onlinopt>
			</digtopt>
		</digform>
			<digform>
				<digtinfo>
					<formname>PNG</formname>
					<formvern>1</formvern>
					<formspec>USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Raster_EqTotalCounts.png</formspec>
					<formcont>Portable Network Graphics (PNG) raster image associated with the data set.</formcont>
					<filedec>No compression

# Populate Metadata Template

In [171]:
#Load XML Metadata Template File and Read It
metaData = os.path.join(radMetaDataTemplatePath, radMetaDataTemplateName)
print ('Metadata path: ' + radMetaDataTemplatePath + '\n')
xmlTemplateFile = open(metaData, 'r')
metaDataContent = xmlTemplateFile.readlines()
print(metaDataContent)
xmlTemplateFile.close()


Metadata path: C:\CurrentWork\DataReleases\NorthwesternArkansas\Metadata

['<?xml version="1.0" encoding="UTF-8"?>\n', '<metadata>\n', '  <idinfo>\n', '    <citation>\n', '      <citeinfo>{origin}\n', '        <pubdate>{pubdate}</pubdate>\n', '        <title>{title}</title>\n', '        <edition>1</edition>\n', '        <geoform>ASCII and Binary Digital Data</geoform>\n', '        <pubinfo>\n', '          <pubplace>Denver, CO</pubplace>\n', '          <publish>U.S. Geological Survey</publish>\n', '        </pubinfo>\n', '        <othercit>{othercit}</othercit>\n', '        <onlink>{onlink}</onlink>\n', '      </citeinfo>\n', '    </citation>\n', '    <descript>\n', '      <abstract>{abstract}</abstract>\n', '      <purpose>{purpose}</purpose>\n', '      <supplinf>{supplinf}</supplinf>\n', '    </descript>\n', '    <timeperd>\n', '      <timeinfo>\n', '        <rngdates>\n', '          <begdate>{begdate}</begdate>\n', '          <enddate>{enddate}</enddate>\n', '        </rngdates>\n', 

### !!! Be sure to enter output file name for the metadata below !!!

In [172]:
# Please enter metadata output file name below between the single quotes
#========================================================================================================================
ragMetadataOutputFileName = 'USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Radiometric.xml'
#========================================================================================================================

In [173]:
# Replace values of current metadata template with the appropriate values.  
# All of this input should have been defined when going through the steps outlined above.
lineString = ''
#newMetaDataContent = metaDataContent
myfilename = radMetaDataTemplatePath + "\\" + ragMetadataOutputFileName
xmlFile = open(myfilename,"w+")
print('File exported to:\n'+ myfilename)
#print(keywords.value)
for i in range(len(metaDataContent)):
    lineString = metaDataContent[i]
    if lineString.find('{origin}'):
     lineString = lineString.replace('{origin}',origin)
    
    if lineString.find('{pubdate}'):
     lineString = lineString.replace('{pubdate}', pubdate)    
    
    if lineString.find('{title}'):
     lineString = lineString.replace('{title}', title)
    
    if lineString.find('{othercit}'):
     lineString = lineString.replace('{othercit}', othercit)
    
    if lineString.find('{onlink}'):
     lineString = lineString.replace('{onlink}', onlink)
    
    if lineString.find('{abstract}'):
     lineString = lineString.replace('{abstract}', abstract)
    
    if lineString.find('{purpose}'):
     lineString = lineString.replace('{purpose}', purpose)
    
    if lineString.find('{supplinf}'):
     lineString = lineString.replace('{supplinf}', supplinf)
    
    if lineString.find('{begdate}'):
     lineString = lineString.replace('{begdate}', begdate)
    
    if lineString.find('{enddate}'):
     lineString = lineString.replace('{enddate}', enddate)
    
    if lineString.find('{westbc}'):
     lineString = lineString.replace('{westbc}', str(westbc))

    if lineString.find('{eastbc}'):
     lineString = lineString.replace('{eastbc}', str(eastbc))

    if lineString.find('{northbc}'):
     lineString = lineString.replace('{northbc}', str(northbc))
    
    if lineString.find('{southbc}'):
     lineString = lineString.replace('{southbc}', str(southbc))
   
    if lineString.find('{keywords}'):
     lineString = lineString.replace('{keywords}', str(keywords))
    
    if lineString.find('{accconst}'):
     lineString = lineString.replace('{accconst}', accconst)
    
    if lineString.find('{useconst}'):
     lineString = lineString.replace('{useconst}', useconst)
    
    if lineString.find('{cntperDataContact}'):
     lineString = lineString.replace('{cntperDataContact}', cntperDataContact)
    
    if lineString.find('{cntposDataContact}'):
     lineString = lineString.replace('{cntposDataContact}', cntposDataContact)
    
    if lineString.find('{cntvoiceDataContact}'):
     lineString = lineString.replace('{cntvoiceDataContact}', str(cntvoiceDataContact))
    
    if lineString.find('{cntemailDataContact}'):
     lineString = lineString.replace('{cntemailDataContact}', str(cntemailDataContact))
    
    if lineString.find('{datacred}'):
     lineString = lineString.replace('{datacred}', str(datacred))
    
    if lineString.find('{attraccr}'):
     lineString = lineString.replace('{attraccr}', str(attraccr)) 
    
    if lineString.find('{logic}'):
     lineString = lineString.replace('{logic}', logic)
    
    if lineString.find('{complete}'):
     lineString = lineString.replace('{complete}', str(complete))
    
    if lineString.find('{horizpar}'):
     lineString = lineString.replace('{horizpar}', str(horizpar))
    
    if lineString.find('{vertaccr}'):
     lineString = lineString.replace('{vertaccr}', str(vertaccr))
    
    if lineString.find('{lineage}'):
     lineString = lineString.replace('{lineage}', str(lineage))
    
    if lineString.find('{latres}'):
     lineString = lineString.replace('{latres}', str(latres))
    
    if lineString.find('{longres}'):
     lineString = lineString.replace('{longres}', str(longres))
    
    if lineString.find('{horizdn}'):
     lineString = lineString.replace('{horizdn}', str(horizdn))
    
    if lineString.find('{ellips}'):
     lineString = lineString.replace('{ellips}', str(ellips))
    
    if lineString.find('{semiaxis}'):
     lineString = lineString.replace('{semiaxis}', str(semiaxis))
    
    if lineString.find('{denflat}'):
     lineString = lineString.replace('{denflat}', str(denflat))
   
    if lineString.find('{eainfo}'):
     lineString = lineString.replace('{eainfo}', str(eainfo))
    
    if lineString.find('{distliab}'):
     lineString = lineString.replace('{distliab}', str(distliab))
    
    if lineString.find('{metd}'):
     lineString = lineString.replace('{metd}', str(metd))
    
    if lineString.find('{cntperMetaInfo}'):
     lineString = lineString.replace('{cntperMetaInfo}', str(cntperMetaInfo))
    
    if lineString.find('{cntposMetaInfo}'):
     lineString = lineString.replace('{cntposMetaInfo}', str(cntposMetaInfo))
    
    if lineString.find('{cntvoiceMetaInfo}'):
     lineString = lineString.replace('{cntvoiceMetaInfo}', str(cntvoiceMetaInfo))
    
    if lineString.find('{cntemailMetaInfo}'):
     lineString = lineString.replace('{cntemailMetaInfo}', str(cntemailMetaInfo))    
    
    if lineString.find('{stdorder}'):
     lineString = lineString.replace('{stdorder}', str(stdorder))
    
    if lineString.find('{metd}'):
     lineString = lineString.replace('{metd}', str(metd))
    
    if lineString.find('{cntperMetaInfo}'):
     lineString = lineString.replace('{cntperMetaInfo}', str(cntperMetaInfo))
    
    if lineString.find('{cntposMetaInfo}'):
     lineString = lineString.replace('{cntposMetaInfo}', str(cntposMetaInfo))
    
    if lineString.find('{cntvoiceMetaInfo}'):
     lineString = lineString.replace('{cntvoiceMetaInfo}', str(cntvoiceMetaInfo))
    
    if lineString.find('{cntemailMetaInfo}'):
     lineString = lineString.replace('{cntemailMetaInfo}', str(cntemailMetaInfo))

    else:
     lineString = lineString
    xmlFile.write(lineString)
    
    #print (lineString)
     
    
    
#for r in (metaDataContent):
    #newMetaDataContent = metaDataContent.replace('{title}', drTitle)
    #newMetaDataContent = metaDataContent.replace('{keywords}', keywords.value)
xmlFile.close()

print ('Creation of new metadata file is complete\n\n') 
#Load EDI File and Read It
##checkFile = open(open(myfilename, 'r')
##checkFileContent = checkFile.read()
##checkFile.close()
##print(checkFileContent)

File exported to:
C:\CurrentWork\DataReleases\NorthwesternArkansas\Metadata\USA_Northwest-Arkansas-Airborne-Survey-2019-2020_Radiometric.xml
Creation of new metadata file is complete




### Check this file to see if it is valid against the FGDC metadata standard (FGDC-STD-001-1998)

## https://mrdata.usgs.gov/validation/